In [2]:
"""This is a simple example demonstrating how to clone the behavior of an expert.

Refer to the jupyter notebooks for more detailed examples of how to use the algorithms.
"""
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

/home/foxy_user/venvs/thesis_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rng = np.random.default_rng(0)
env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=rng,
    post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],  # for computing rollouts
)

In [4]:
def train_expert():
    # note: use `download_expert` instead to download a pretrained, competent expert
    print("Training a expert.")
    expert = PPO(
        policy=MlpPolicy,
        env=env,
        seed=0,
        batch_size=64,
        ent_coef=0.0,
        learning_rate=0.0003,
        n_epochs=10,
        n_steps=64,
    )
    expert.learn(1_000)  # Note: change this to 100_000 to train a decent expert.
    return expert


def download_expert():
    print("Downloading a pretrained expert.")
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals-CartPole-v0",
        venv=env,
    )
    return expert


def sample_expert_transitions():
    # expert = train_expert()  # uncomment to train your own expert
    expert = download_expert()

    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=None, min_episodes=50),
        rng=rng,
    )
    return rollout.flatten_trajectories(rollouts)

In [5]:
transitions = sample_expert_transitions()

ppo-seals-CartPole-v0.zip: 100%|██████████| 139k/139k [00:00<00:00, 3.64MB/s]


Sampling expert transitions.


In [12]:
print(transitions[0:2]) 


Transitions(obs=array([[ 0.0043625 ,  0.04350724,  0.03158535, -0.04972615],
       [ 0.00523264,  0.23816238,  0.03059083, -0.33227867]],
      dtype=float32), acts=array([1, 0]), infos=array([{}, {}], dtype=object), next_obs=array([[ 0.00523264,  0.23816238,  0.03059083, -0.33227867],
       [ 0.00999589,  0.04261867,  0.02394526, -0.03010804]],
      dtype=float32), dones=array([False, False]))


In [22]:
from imitation.data.types import Trajectory
no_of_transitions = 10
obs=np.ones(shape=(no_of_transitions,3))*3
acts=np.ones(shape=(no_of_transitions-1,1))*5
next_obs=np.ones(shape=(no_of_transitions,3))*7
dones=np.zeros(shape=(no_of_transitions,1))
infos= [{} for i in range(no_of_transitions-1)]
custom_dataset = Trajectory(obs=obs, acts=acts,infos=infos,terminal=dones)

rollout2= rollout.flatten_trajectories(custom_dataset)
print(rollout2[0:2])

TypeError: 'Trajectory' object is not iterable

In [ ]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

evaluation_env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=rng,
    env_make_kwargs={"render_mode": "human"},  # for rendering
)

print("Evaluating the untrained policy.")
reward, _ = evaluate_policy(
    bc_trainer.policy,  # type: ignore[arg-type]
    evaluation_env,
    n_eval_episodes=3,
    render=True,  # comment out to speed up
)
print(f"Reward before training: {reward}")

print("Training a policy using Behavior Cloning")
bc_trainer.train(n_epochs=1)

print("Evaluating the trained policy.")
reward, _ = evaluate_policy(
    bc_trainer.policy,  # type: ignore[arg-type]
    evaluation_env,
    n_eval_episodes=3,
    render=True,  # comment out to speed up
)
print(f"Reward after training: {reward}")